In [12]:
import mysql.connector

# Establish a connection to the MySQL database
connection = mysql.connector.connect(
    host="falode3368-1.ch6kyu62mcnp.us-east-1.rds.amazonaws.com",
    user="Falode",
    password="1QAZ2wsx",
    database="cis3368DB"
)

# Create a cursor object to execute SQL queries
cursor = connection.cursor()

# Define SQL statements to create tables
create_facility_table = """
CREATE TABLE IF NOT EXISTS facility (
    id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(255)
)
"""

create_classroom_table = """
CREATE TABLE IF NOT EXISTS classroom (
    id INT AUTO_INCREMENT PRIMARY KEY,
    capacity INT,
    name VARCHAR(255),
    facility_id INT,
    CONSTRAINT fk_facility_id FOREIGN KEY (facility_id) REFERENCES facility(id)
)
"""

create_teacher_table = """
CREATE TABLE IF NOT EXISTS teacher (
    id INT AUTO_INCREMENT PRIMARY KEY,
    firstname VARCHAR(255),
    lastname VARCHAR(255),
    classroom_id INT,
    CONSTRAINT fk_classroom_id FOREIGN KEY (classroom_id) REFERENCES classroom(id)
)
"""

create_child_table = """
CREATE TABLE IF NOT EXISTS child (
    id INT AUTO_INCREMENT PRIMARY KEY,
    firstname VARCHAR(255),
    lastname VARCHAR(255),
    age INT,
    classroom_id INT,
    CONSTRAINT fk_classroom_id_child FOREIGN KEY (classroom_id) REFERENCES classroom(id)
)
"""

# Execute the SQL statements to create tables
cursor.execute(create_facility_table)
cursor.execute(create_classroom_table)
cursor.execute(create_teacher_table)
cursor.execute(create_child_table)

# Insert 15 entries into the facility table
for i in range(1, 16):
    cursor.execute(f"INSERT INTO facility (name) VALUES ('Facility {i}')")

# Insert 15 entries into the classroom table
for i in range(1, 16):
    cursor.execute(f"INSERT INTO classroom (capacity, name, facility_id) VALUES (20, 'Classroom {i}', 1)")

# Insert 15 entries into the teacher table
for i in range(1, 16):
    cursor.execute(f"INSERT INTO teacher (firstname, lastname, classroom_id) VALUES ('Teacher {i}', 'Lastname {i}', 1)")

# Insert 15 entries into the child table
for i in range(1, 16):
    cursor.execute(f"INSERT INTO child (firstname, lastname, age, classroom_id) VALUES ('Child {i}', 'Lastname {i}', 5, 1)")

# Commit the changes and close the connection
connection.commit()
connection.close()

print("Tables created and entries inserted successfully!")


Tables created and entries inserted successfully!


In [13]:
import mysql.connector

# Establish a connection to the MySQL database
connection = mysql.connector.connect(
    host="falode3368-1.ch6kyu62mcnp.us-east-1.rds.amazonaws.com",
    user="Falode",
    password="1QAZ2wsx",
    database="cis3368DB"
)

# Create a cursor object to execute SQL queries
cursor = connection.cursor()

# Define SQL statements to select data from tables
select_facility_table = "SELECT * FROM facility"
select_classroom_table = "SELECT * FROM classroom"
select_teacher_table = "SELECT * FROM teacher"
select_child_table = "SELECT * FROM child"

# Execute the SQL statements to select data from tables
cursor.execute(select_facility_table)
facility_data = cursor.fetchall()

cursor.execute(select_classroom_table)
classroom_data = cursor.fetchall()

cursor.execute(select_teacher_table)
teacher_data = cursor.fetchall()

cursor.execute(select_child_table)
child_data = cursor.fetchall()

# Print the data from each table
print("Facility table:")
for row in facility_data:
    print(row)

print("\nClassroom table:")
for row in classroom_data:
    print(row)

print("\nTeacher table:")
for row in teacher_data:
    print(row)

print("\nChild table:")
for row in child_data:
    print(row)

# Close the connection
connection.close()


Facility table:
(1, 'Facility 1')
(2, 'Facility 2')
(3, 'Facility 3')
(4, 'Facility 4')
(5, 'Facility 5')
(6, 'Facility 6')
(7, 'Facility 7')
(8, 'Facility 8')
(9, 'Facility 9')
(10, 'Facility 10')
(11, 'Facility 11')
(12, 'Facility 12')
(13, 'Facility 13')
(14, 'Facility 14')
(15, 'Facility 15')

Classroom table:
(1, 20, 'Classroom 1', 1)
(2, 20, 'Classroom 2', 1)
(3, 20, 'Classroom 3', 1)
(4, 20, 'Classroom 4', 1)
(5, 20, 'Classroom 5', 1)
(6, 20, 'Classroom 6', 1)
(7, 20, 'Classroom 7', 1)
(8, 20, 'Classroom 8', 1)
(9, 20, 'Classroom 9', 1)
(10, 20, 'Classroom 10', 1)
(11, 20, 'Classroom 11', 1)
(12, 20, 'Classroom 12', 1)
(13, 20, 'Classroom 13', 1)
(14, 20, 'Classroom 14', 1)
(15, 20, 'Classroom 15', 1)

Teacher table:
(1, 'Teacher 1', 'Lastname 1', 1)
(2, 'Teacher 2', 'Lastname 2', 1)
(3, 'Teacher 3', 'Lastname 3', 1)
(4, 'Teacher 4', 'Lastname 4', 1)
(5, 'Teacher 5', 'Lastname 5', 1)
(6, 'Teacher 6', 'Lastname 6', 1)
(7, 'Teacher 7', 'Lastname 7', 1)
(8, 'Teacher 8', 'Lastname 8

In [8]:
from flask import Flask, request, jsonify
import mysql.connector

app = Flask(__name__)

# Database connection
connection = mysql.connector.connect(
    host="falode3368-1.ch6kyu62mcnp.us-east-1.rds.amazonaws.com",
    user="Falode",
    password="1QAZ2wsx",
    database="cis3368DB"
)
cursor = connection.cursor()

# Function to assign teacher to classroom
def assign_teacher_to_classroom(teacher_id, classroom_id):
    # Check the number of children already assigned to the classroom
    cursor.execute("SELECT COUNT(*) FROM child WHERE classroom_id = %s", (classroom_id,))
    num_children = cursor.fetchone()[0]

    # Check the number of teachers assigned to the classroom
    cursor.execute("SELECT COUNT(*) FROM teacher WHERE classroom_id = %s", (classroom_id,))
    num_teachers = cursor.fetchone()[0]

    # Calculate the adjusted capacity based on the number of teachers
    adjusted_capacity = classroom_capacity - num_teachers * 10

    # Check if the teacher assignment exceeds the adjusted capacity
    if num_children >= adjusted_capacity:
        return False

    # If capacity allows, assign the teacher to the classroom
    cursor.execute("UPDATE teacher SET classroom_id = %s WHERE id = %s", (classroom_id, teacher_id))
    connection.commit()
    return True

# Function to assign child to classroom
def assign_child_to_classroom(child_id, classroom_id):
    # Check the number of teachers assigned to the classroom
    cursor.execute("SELECT COUNT(*) FROM teacher WHERE classroom_id = %s", (classroom_id,))
    num_teachers = cursor.fetchone()[0]

    # Check if there are teachers assigned to the classroom
    if num_teachers == 0:
        return False

    # Check the number of children already assigned to the classroom
    cursor.execute("SELECT COUNT(*) FROM child WHERE classroom_id = %s", (classroom_id,))
    num_children = cursor.fetchone()[0]

    # Calculate the adjusted capacity based on the number of teachers
    adjusted_capacity = classroom_capacity - num_teachers * 10

    # Check if the child assignment exceeds the adjusted capacity
    if num_children >= adjusted_capacity:
        return False

    # If capacity allows, assign the child to the classroom
    cursor.execute("UPDATE child SET classroom_id = %s WHERE id = %s", (classroom_id, child_id))
    connection.commit()
    return True

# Route to add a new teacher
@app.route('/teacher', methods=['POST'])
def add_teacher():
    data = request.json
    firstname = data['firstname']
    lastname = data['lastname']
    classroom_id = data['classroom_id']

    cursor.execute("INSERT INTO teacher (firstname, lastname, classroom_id) VALUES (%s, %s, %s)", (firstname, lastname, classroom_id))
    connection.commit()
    return jsonify({'message': 'Teacher added successfully'})

# Route to add a new child
@app.route('/child', methods=['POST'])
def add_child():
    data = request.json
    firstname = data['firstname']
    lastname = data['lastname']
    age = data['age']
    classroom_id = data['classroom_id']

    cursor.execute("INSERT INTO child (firstname, lastname, age, classroom_id) VALUES (%s, %s, %s, %s)", (firstname, lastname, age, classroom_id))
    connection.commit()
    return jsonify({'message': 'Child added successfully'})

# Route to assign teacher to classroom
@app.route('/assign_teacher', methods=['POST'])
def assign_teacher():
    data = request.json
    teacher_id = data['teacher_id']
    classroom_id = data['classroom_id']

    if assign_teacher_to_classroom(teacher_id, classroom_id):
        return jsonify({'message': 'Teacher assigned to classroom successfully'})
    else:
        return jsonify({'error': 'Teacher assignment failed. Classroom capacity exceeded'})

# Route to assign child to classroom
@app.route('/assign_child', methods=['POST'])
def assign_child():
    data = request.json
    child_id = data['child_id']
    classroom_id = data['classroom_id']

    if assign_child_to_classroom(child_id, classroom_id):
        return jsonify({'message': 'Child assigned to classroom successfully'})
    else:
        return jsonify({'error': 'Child assignment failed. No teachers assigned to the classroom'})

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat
Traceback (most recent call last):
  File "/home/dragon/.local/lib/python3.8/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/dragon/.local/lib/python3.8/site-packages/traitlets/config/application.py", line 1074, in launch_instance
    app.initialize(argv)
  File "/home/dragon/.local/lib/python3.8/site-packages/traitlets/config/application.py", line 118, in inner
    return method(app, *args, **kwargs)
  File "/home/dragon/.local/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 692, in initialize
    self.init_sockets()
  File "/home/dragon/.local/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 331, in init_sockets
    self.shell_port = self._bind_socket(self.shell_socket, self.shell_port)
  File "/home/dragon/.local/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 253, in _bind_socket
    return self._try_bind_socket(s, port

SystemExit: 1

/home/dragon/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
import mysql.connector
from flask import Flask, request, jsonify

# Database connection
connection = mysql.connector.connect(
    host="falode3368-1.ch6kyu62mcnp.us-east-1.rds.amazonaws.com",
    user="Falode",
    password="1QAZ2wsx",
    database="cis3368DB"
)

app = Flask(__name__)

# Route for adding a new facility
@app.route('/facility', methods=['POST'])
def add_facility():
    data = request.json
    name = data.get('name')
    if name:
        cursor = connection.cursor()
        cursor.execute("INSERT INTO facility (name) VALUES (%s)", (name,))
        connection.commit()
        cursor.close()
        return jsonify({'message': 'Facility added successfully'}), 201
    else:
        return jsonify({'error': 'Name is required'}), 400

# Route for adding a new teacher
@app.route('/teacher', methods=['POST'])
def add_teacher():
    data = request.json
    firstname = data.get('firstname')
    lastname = data.get('lastname')
    classroom_id = data.get('classroom_id')
    if firstname and lastname and classroom_id:
        cursor = connection.cursor()
        cursor.execute("INSERT INTO teacher (firstname, lastname, classroom_id) VALUES (%s, %s, %s)", (firstname, lastname, classroom_id))
        connection.commit()
        cursor.close()
        return jsonify({'message': 'Teacher added successfully'}), 201
    else:
        return jsonify({'error': 'Firstname, lastname, and classroom_id are required'}), 400

# Route for adding a new child
@app.route('/child', methods=['POST'])
def add_child():
    data = request.json
    firstname = data.get('firstname')
    lastname = data.get('lastname')
    age = data.get('age')
    classroom_id = data.get('classroom_id')
    if firstname and lastname and age and classroom_id:
        cursor = connection.cursor()
        cursor.execute("INSERT INTO child (firstname, lastname, age, classroom_id) VALUES (%s, %s, %s, %s)", (firstname, lastname, age, classroom_id))
        connection.commit()
        cursor.close()
        return jsonify({'message': 'Child added successfully'}), 201
    else:
        return jsonify({'error': 'Firstname, lastname, age, and classroom_id are required'}), 400

# Route for assigning a child to a teacher
@app.route('/assign', methods=['POST'])
def assign_child():
    data = request.json
    child_id = data.get('child_id')
    teacher_id = data.get('teacher_id')
    if child_id and teacher_id:
        cursor = connection.cursor()
        cursor.execute("SELECT * FROM child WHERE id = %s", (child_id,))
        child = cursor.fetchone()
        cursor.execute("SELECT * FROM teacher WHERE id = %s", (teacher_id,))
        teacher = cursor.fetchone()
        if child and teacher:
            cursor.execute("UPDATE child SET teacher_id = %s WHERE id = %s", (teacher_id, child_id))
            connection.commit()
            cursor.close()
            return jsonify({'message': 'Child assigned to teacher successfully'}), 200
        else:
            return jsonify({'error': 'Child or teacher not found'}), 404
    else:
        return jsonify({'error': 'Child_id and teacher_id are required'}), 400

# Route for deleting a facility
@app.route('/facility/<int:facility_id>', methods=['DELETE'])
def delete_facility(facility_id):
    cursor = connection.cursor()
    cursor.execute("DELETE FROM facility WHERE id = %s", (facility_id,))
    connection.commit()
    cursor.close()
    return jsonify({'message': 'Facility deleted successfully'}), 200

# Add other routes for CRUD operations...

if __name__ == '__main__':
    app.run(debug=True)
